In [1]:
import numpy as np
import pandas as pd
import re
from collections import Counter

In [2]:
def process_data(path):
    with open(path) as f:
        data = f.read()
    data = data.lower()
    words = re.findall('\w+',data)
    return words

In [3]:
total_words = process_data('shakespeare.txt')
# To find unique words just convert it to a set
vocab = set(total_words)

In [4]:
print(len(total_words))
print(len(vocab))

53614
6116


In [5]:
def get_count(words):
    word_dict = Counter(words)
    return word_dict

In [6]:
words_count_dict = get_count(total_words)

In [7]:
def get_probs(words_dict):
    probs = {}
    m = sum(words_dict.values())
    for key in words_dict.keys():
        probs[key] = words_dict[key]/m
    return probs

In [8]:
probs = get_probs(words_count_dict)
probs['thee'] # probability of the word thee

0.004476442720185026

In [9]:
def delete_letter(word):
    splits = [(word[:i],word[i:]) for i in range(len(word))]
    words = [(a+b[1:]) for a,b in splits]
    return words

In [10]:
def switch_letter(word):
    splits = [(word[:i],word[i:]) for i in range(len(word))]
    new_words = []
    for a,b in splits:
        if(len(b)>1):
            new_words.append(a+b[1]+b[0]+b[2:])
    return new_words

In [11]:
def replace_letter(word):
    replace = []
    s = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i],word[i:]) for i in range(len(word))]
    replace = [a + l + (b[1:] if len(b)> 1 else '') for a,b in splits if b for l in s]
    replace_set=set(replace)    
    replace_set.remove(word)
    replaced = sorted(list(replace_set))
    return replaced

In [12]:
def insert_letter(word):
    insert = []
    s = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i],word[i:]) for i in range(len(word)+1)]
    insert = [a+l+b for a,b in splits for l in s]
    return insert

In [13]:
def edit_one_letter(word, allow_switches = True):
    edit_one_set = set() # to remove duplicates
    edit_one_set.update(delete_letter(word))
    if allow_switches:
        edit_one_set.update(switch_letter(word))
    edit_one_set.update(replace_letter(word))
    edit_one_set.update(insert_letter(word))
    return edit_one_set

In [14]:
def edit_two_letters(word, allow_switches = True):
    edit_two_set = set() # to remove duplicates
    edit_one = edit_one_letter(word,allow_switches=allow_switches)
    print(type(edit_one))
    for w in edit_one:
        if w:
            edit_two = edit_one_letter(w,allow_switches=allow_switches)
            edit_two_set.update(edit_two)
    return edit_two_set

In [15]:
def get_words(word,vocab,probs,n=2):
    suggestions = []
    suggestions = list((word in vocab and word) or edit_one_letter(word).intersection(vocab) or edit_two_letters(word).intersection(vocab))
    best = [(s,probs[s]) for s in list(reversed(suggestions))]
    return (best)

In [16]:
get_words('eta',vocab,probs)

[('ta', 1.865184466743761e-05),
 ('eat', 0.00022382213600925132),
 ('etc', 3.730368933487522e-05)]

In [17]:
get_words('dys',vocab,probs)

[('dye', 1.865184466743761e-05), ('days', 0.0004103405826836274)]

In [18]:
def min_edit_distance(source,target,ins_cost=1,del_cost=1,rep_cost=2):
    m = len(source)
    n = len(target)
    D = np.zeros((m+1,n+1)).astype(np.int32)
    
    for i in range(1,(m+1)):
        D[i][0] = D[i-1][0] + del_cost
    for j in range(1,(n+1)):
        D[0][j] = D[0][j-1] + ins_cost
        
    for i in range(1,(m+1)):
        for j in range(1,(n+1)):
            r_cost=rep_cost
            if(source[i-1] == target[j-1]):
                r_cost=0
            D[i][j] = min([D[i-1][j]+del_cost, D[i][j-1]+ins_cost, D[i-1][j-1]+r_cost])
    min_edit = D[m][n]
    return D,min_edit

In [19]:
D,min_edit = min_edit_distance('play','stay')

In [20]:
D

array([[0, 1, 2, 3, 4],
       [1, 2, 3, 4, 5],
       [2, 3, 4, 5, 6],
       [3, 4, 5, 4, 5],
       [4, 5, 6, 5, 4]])

In [21]:
min_edit
# Minimum edits required to convert source to target

4